In [2]:
from imdb import Cinemagoer


In [3]:
# Create an instance of the IMDb class
ia = Cinemagoer()

# Get movie by IMDb ID
movie_id = '0816692'  # This is the IMDb ID for the movie "Interstellar"
movie = ia.get_movie(movie_id)

In [32]:
# Get the first cast member of the movie
first_cast_member = movie['cast'][:20] if 'cast' in movie else None

In [33]:
first_cast_member

[<Person id:0000995[http] name:_Ellen Burstyn_>,
 <Person id:0000190[http] name:_Matthew McConaughey_>,
 <Person id:3237775[http] name:_Mackenzie Foy_>,
 <Person id:0001475[http] name:_John Lithgow_>,
 <Person id:3154303[http] name:_Timothée Chalamet_>,
 <Person id:0654648[http] name:_David Oyelowo_>,
 <Person id:2180792[http] name:_Collette Wolfe_>,
 <Person id:0565133[http] name:_Francis X. McCarthy_>,
 <Person id:0410347[http] name:_Bill Irwin_>,
 <Person id:0004266[http] name:_Anne Hathaway_>,
 <Person id:0095960[http] name:_Andrew Borba_>,
 <Person id:0004747[http] name:_Wes Bentley_>,
 <Person id:0001137[http] name:_William Devane_>,
 <Person id:0000323[http] name:_Michael Caine_>,
 <Person id:1408543[http] name:_David Gyasi_>,
 <Person id:1577637[http] name:_Josh Stewart_>,
 <Person id:0000729[http] name:_Casey Affleck_>,
 <Person id:1489668[http] name:_Leah Cairns_>,
 <Person id:1567113[http] name:_Jessica Chastain_>,
 <Person id:5291430[http] name:_Liam Dickinson_>]

In [35]:
if first_cast_member[18]:
    actor_id = first_cast_member[18].personID
    actor = ia.get_person(actor_id)  # Retrieve full details of the actor
    
    # Extracting various pieces of information
    actor_name = actor.get('name', 'N/A')
    actor_birth_date = actor.get('birth date', 'N/A')
    actor_height = actor.get('height', 'N/A')
    actor_biography = actor.get('biography', ['N/A'])[0]  # IMDb stores biography as a list
    actor_filmography = actor.get('filmography', 'N/A')  # Detailed filmography
    actor_trivia = actor.get('trivia', ['N/A'])[0]  # Trivia is also a list, taking the first entry
    
    actor_photo_url = actor.get('headshot', 'No photo available')
    
    # Print all available details
    print(f"Actor Name: {actor_name}")
    print(f"Birth Date: {actor_birth_date}")
    print(f"Height: {actor_height}")
    print(f"Biography: {actor_biography[:500]}...")  # Limiting biography output
    print(f"Trivia: {actor_trivia}")
    print(f"Filmography: {actor_filmography}")
    print(f"Actor Photo URL: {actor_photo_url}")
else:
    print("No cast member found.")

Actor Name: Jessica Chastain
Birth Date: N/A
Height: N/A
Biography: Jessica Michelle Chastain was born in Sacramento, California, and was raised in a middle-class household in a Northern California suburb. Her mother, Jerri Chastain, is a vegan chef whose family is originally from Kansas, and her stepfather is a fireman. She discovered dance at the age of nine and was in a dance troupe by age thirteen. She began performing in Shakespearean productions all over the Bay area.An actor in a production of "Romeo & Juliet" encouraged her to audition for Juilliard as a...
Trivia: N/A
Filmography: N/A
Actor Photo URL: https://m.media-amazon.com/images/M/MV5BMTU1MDM5NjczOF5BMl5BanBnXkFtZTcwOTY2MDE4OA@@._V1_QL75_UX140_CR0,0,140,207_.jpg


In [36]:
print(actor_biography)

Jessica Michelle Chastain was born in Sacramento, California, and was raised in a middle-class household in a Northern California suburb. Her mother, Jerri Chastain, is a vegan chef whose family is originally from Kansas, and her stepfather is a fireman. She discovered dance at the age of nine and was in a dance troupe by age thirteen. She began performing in Shakespearean productions all over the Bay area.An actor in a production of "Romeo & Juliet" encouraged her to audition for Juilliard as a drama major. She became a member of "Crew 32" with the help of a scholarship from one of the school's famous alumni, 'Robin Williams' (qv).In her last year at Juilliard, she was offered a holding deal with TV writer/producer 'John Wells' (qv) and she eventually worked in three of his TV shows. Jessica continues to do theatre, having played in "The Cherry Orchard", "Rodney's Wife", "Salome" and "Othello". She spends her time between New York and Los Angeles, working in theater, film and TV.In 20